In [18]:
# All code from Tutorial:
# https://www.machinelearningplus.com/nlp/topic-modeling-gensim-python/

# Part 1 : gensim LDA based on NLTK & SpaCy

# Run in python console
import nltk; nltk.download('stopwords')


[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/tharsen/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [19]:
import re
import numpy as np
import pandas as pd
from pprint import pprint

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# spacy for lemmatization
import spacy

# Plotting tools
import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline

# Enable logging for gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

In [20]:
# NLTK Stop words
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use'])


In [21]:
# Import Dataset
df = pd.read_json('https://raw.githubusercontent.com/selva86/datasets/master/newsgroups.json')
print(df.target_names.unique())
df.head()

# Convert to list
data = df.content.values.tolist()

# Remove Emails
data = [re.sub('\S*@\S*\s?', '', sent) for sent in data]

# Remove new line characters
data = [re.sub('\s+', ' ', sent) for sent in data]

# Remove distracting single quotes
data = [re.sub("\'", "", sent) for sent in data]

pprint(data[:1])

['rec.autos' 'comp.sys.mac.hardware' 'rec.motorcycles' 'misc.forsale'
 'comp.os.ms-windows.misc' 'alt.atheism' 'comp.graphics'
 'rec.sport.baseball' 'rec.sport.hockey' 'sci.electronics' 'sci.space'
 'talk.politics.misc' 'sci.med' 'talk.politics.mideast'
 'soc.religion.christian' 'comp.windows.x' 'comp.sys.ibm.pc.hardware'
 'talk.politics.guns' 'talk.religion.misc' 'sci.crypt']
['From: (wheres my thing) Subject: WHAT car is this!? Nntp-Posting-Host: '
 'rac3.wam.umd.edu Organization: University of Maryland, College Park Lines: '
 '15 I was wondering if anyone out there could enlighten me on this car I saw '
 'the other day. It was a 2-door sports car, looked to be from the late 60s/ '
 'early 70s. It was called a Bricklin. The doors were really small. In '
 'addition, the front bumper was separate from the rest of the body. This is '
 'all I know. If anyone can tellme a model name, engine specs, years of '
 'production, where this car is made, history, or whatever info you have on '
 't

In [22]:
%%time

def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

data_words = list(sent_to_words(data))

print(data_words[:1])
print('\n')

[['from', 'wheres', 'my', 'thing', 'subject', 'what', 'car', 'is', 'this', 'nntp', 'posting', 'host', 'rac', 'wam', 'umd', 'edu', 'organization', 'university', 'of', 'maryland', 'college', 'park', 'lines', 'was', 'wondering', 'if', 'anyone', 'out', 'there', 'could', 'enlighten', 'me', 'on', 'this', 'car', 'saw', 'the', 'other', 'day', 'it', 'was', 'door', 'sports', 'car', 'looked', 'to', 'be', 'from', 'the', 'late', 'early', 'it', 'was', 'called', 'bricklin', 'the', 'doors', 'were', 'really', 'small', 'in', 'addition', 'the', 'front', 'bumper', 'was', 'separate', 'from', 'the', 'rest', 'of', 'the', 'body', 'this', 'is', 'all', 'know', 'if', 'anyone', 'can', 'tellme', 'model', 'name', 'engine', 'specs', 'years', 'of', 'production', 'where', 'this', 'car', 'is', 'made', 'history', 'or', 'whatever', 'info', 'you', 'have', 'on', 'this', 'funky', 'looking', 'car', 'please', 'mail', 'thanks', 'il', 'brought', 'to', 'you', 'by', 'your', 'neighborhood', 'lerxst']]


CPU times: user 10.5 s, sys

In [23]:
%%time

# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

# See trigram example
print(trigram_mod[bigram_mod[data_words[0]]])
print('\n')

['from', 'wheres', 'my', 'thing', 'subject', 'what', 'car', 'is', 'this', 'nntp_posting_host', 'rac_wam_umd_edu', 'organization', 'university', 'of', 'maryland_college_park', 'lines', 'was', 'wondering', 'if', 'anyone', 'out', 'there', 'could', 'enlighten', 'me', 'on', 'this', 'car', 'saw', 'the', 'other', 'day', 'it', 'was', 'door', 'sports', 'car', 'looked', 'to', 'be', 'from', 'the', 'late', 'early', 'it', 'was', 'called', 'bricklin', 'the', 'doors', 'were', 'really', 'small', 'in', 'addition', 'the', 'front_bumper', 'was', 'separate', 'from', 'the', 'rest', 'of', 'the', 'body', 'this', 'is', 'all', 'know', 'if', 'anyone', 'can', 'tellme', 'model', 'name', 'engine', 'specs', 'years', 'of', 'production', 'where', 'this', 'car', 'is', 'made', 'history', 'or', 'whatever', 'info', 'you', 'have', 'on', 'this', 'funky', 'looking', 'car', 'please', 'mail', 'thanks', 'il', 'brought', 'to', 'you', 'by', 'your', 'neighborhood', 'lerxst']


CPU times: user 46.3 s, sys: 486 ms, total: 46.8 s
Wa

In [24]:
# Define functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [25]:
%%time
# This cell takes 2-3 minutes to run on my machine.  -j

# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# python3 -m spacy download en
nlp = spacy.load('en', disable=['parser', 'ner'])

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

print(data_lemmatized[:1])
print('\n')

[['where', 's', 'thing', 'car', 'nntp_post', 'host', 'rac_wam', 'umd', 'organization', 'university', 'maryland_college', 'park', 'line', 'wonder', 'anyone', 'could', 'enlighten', 'car', 'see', 'day', 'door', 'sport', 'car', 'look', 'late', 'early', 'call', 'bricklin', 'door', 'really', 'small', 'addition', 'front_bumper', 'separate', 'rest', 'body', 'know', 'anyone', 'tellme', 'model', 'name', 'engine', 'spec', 'year', 'production', 'car', 'make', 'history', 'whatev', 'info', 'funky', 'look', 'car', 'mail', 'thank', 'bring', 'neighborhood', 'lerxst']]


CPU times: user 7min 20s, sys: 1min 19s, total: 8min 39s
Wall time: 2min 25s


In [26]:
# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
print(corpus[:1])

[[(0, 1), (1, 2), (2, 1), (3, 1), (4, 1), (5, 1), (6, 5), (7, 1), (8, 1), (9, 2), (10, 1), (11, 1), (12, 1), (13, 1), (14, 1), (15, 1), (16, 1), (17, 1), (18, 1), (19, 1), (20, 1), (21, 1), (22, 2), (23, 1), (24, 1), (25, 1), (26, 1), (27, 1), (28, 1), (29, 1), (30, 1), (31, 1), (32, 1), (33, 1), (34, 1), (35, 1), (36, 1), (37, 1), (38, 1), (39, 1), (40, 1), (41, 1), (42, 1), (43, 1), (44, 1), (45, 1), (46, 1), (47, 1), (48, 1), (49, 1), (50, 1)]]


In [27]:
mywords = []
for cp in corpus:
    for id, freq in cp:
        if id2word[id].find("_") != -1:
            mywords.append(id2word[id]) 
print(len(mywords))
print('\n')
pprint(mywords)

70469


['front_bumper',
 'maryland_college',
 'nntp_post',
 'rac_wam',
 'nntp_post',
 'carson_washington',
 'floppy_disk',
 'guy_kuo',
 'heat_sink',
 'oil_leak',
 'richardson_tx',
 'nntp_post',
 'developers_toolkit',
 'distribution_usa',
 'host_magnusug',
 'magnus_ac',
 'ohio_state',
 'delivered_gateway',
 'diamond_ss',
 'latest_driver',
 'mouse_cursor',
 'nntp_post',
 'automatic_transmission',
 'chris_silvester',
 'james_callison',
 'texas_instrument',
 'nntp_post',
 'ohio_state',
 'acs_ohio',
 'asking_shipping',
 'bottom_magnus',
 'nntp_post',
 'evanston_illinois',
 'northwestern_university',
 'bob_beauchaine',
 'bronx_away',
 'death_penalty',
 'innocents_die',
 'queens_could',
 'robert_beauchaine',
 'sank_manhattan',
 'success_rate',
 'nntp_post',
 'distribution_world',
 'frank_odwyer',
 'host_solntze',
 'jon_livesey',
 'wpd_sgi',
 'nntp_post',
 'dsto_gov',
 'tv_station',
 'west_coast',
 'years_ago',
 'bob_gaj',
 'bob_gajarsky',
 'curtis_jackson',
 'flame_war',
 'lotsa_point',
 'mo

 'distribution_world',
 'scott_roby',
 'apr_gmt',
 'news_reader',
 'administrative_computing',
 'batf_fbi',
 'conspiracy_theorie',
 'emmet_gil',
 'involve_complicat',
 'levine_triumph',
 'man_rik',
 'murders_almost',
 'northern_ireland',
 'student_billing',
 'unlikely_scenarios',
 'waco_today',
 'computing_center',
 'human_right',
 'asia_minor',
 'bosnia_serbia',
 'equal_rights',
 'greece_armenia',
 'ither_alway',
 'mr_tamamidis',
 'ottoman_empire',
 'territorial_water',
 'th_century',
 'turkish_minority',
 'visa_requirement',
 'nntp_post',
 'san_jose',
 'ken_snyder',
 'posters_view',
 'rchland_ibm',
 'nntp_post',
 'electrical_engineer',
 'distribution_na',
 'computer_science',
 'electrical_engineer',
 'cd_rom',
 'distribution_usa',
 '_',
 'ak_q',
 'c__c',
 'c__cx',
 'c__sc',
 'c__scx',
 'ck_chz',
 'ck_ri',
 'cscx_sy',
 'csyx_g',
 'cx_c',
 'cx_cx',
 'cx_cx_s',
 'cx_cx_scx',
 'cx_g',
 'cx_gc',
 'cx_gcucs',
 'cx_gcx',
 'cx_gcx_s',
 'cx_gyx_s',
 'cx_ocx',
 'cx_s',
 'cx_sc',
 'cx_sct',
 'c

 'chinese_food',
 'chinese_restaurant',
 'doug_bank',
 'msg_sensitivity',
 'pulled_muscle',
 'double_blind',
 'advisory_panel',
 'anecdotal_evidence',
 'glutamate_sensitivity',
 'land_mobile',
 'msg_monosodium',
 'products_sector',
 'sector_schaumburg',
 'greatly_appreciat',
 'contact_lens',
 'nntp_post',
 'real_estate',
 'bell_northern',
 'amir_rosenblatt',
 'sam_zbib',
 'newsreader_tin',
 'works_fine',
 'bowling_green',
 'george_pavlic',
 'lori_iannamico',
 'radio_station',
 'ted_simmon',
 'purdue_university',
 'jon_ogden',
 'timothy_brent',
 'mathematical_science',
 'ive_seen',
 'jim_frost',
 'dumbest_automotive',
 'access_unix',
 'itc_uva',
 'stuff_delet',
 'distribution_usa',
 'excellent_condition',
 'public_domain',
 'regression_test',
 'nntp_post',
 'ati_ultra',
 'gateway_dx',
 'hard_drive',
 'ati_gup',
 'color_palette',
 'keith_mancus',
 'palo_alto',
 'distribution_usa',
 'chicago_academic',
 'american_express',
 'andy_freeman',
 'carry_conceal',
 'carry_concealed',
 'concealed

 'hard_disk',
 'hard_drive',
 'nasa_ame',
 'maxtor_lxt',
 'opinions_express',
 'georgia_institute',
 'nntp_post',
 'floppy_drive',
 'nntp_post',
 'sgi_com',
 'silicon_graphic',
 'wait_til',
 'jim_lefebvre',
 'richard_casare',
 'nntp_post',
 'les_bartel',
 'urbana_champaign',
 'fast_polygon',
 'lucas_adamski',
 'nntp_post',
 'computer_science',
 'ftp_site',
 'public_domain',
 'content_type',
 'frequently_ask',
 'ms_do',
 'archive_name',
 'sci_crypt',
 'marc_vanheyningen',
 'silver_ucs',
 'fri_apr',
 'getting_access',
 'gnu_emac',
 'mark_riordan',
 'permanent_resident',
 'privacy_enhanc',
 'readme_file',
 'universally_accept',
 'users_guide',
 'via_anonymous',
 'cl_msu',
 'cryptographic_technique',
 'misc_legal',
 'privacy_enhanced',
 'nntp_post',
 'greatly_appreciat',
 'same_screen',
 'workspace_manager',
 'nntp_post',
 'low_level',
 'ether_twist',
 'old_fashion',
 '_',
 'gamma_ray',
 'distribution_usa',
 'bell_lab',
 'ronald_deblock',
 'netcom_online',
 'anywhere_near',
 'mark_singer',

 'united_stat',
 'sexual_behavior',
 'washington_dc',
 'denver_dept',
 'health_care',
 'volume_number',
 'adult_safety',
 'associate_professor',
 'birth_defect',
 'cdc_surveillance',
 'child_restraint',
 'david_dodell',
 'dhhs_publication',
 'gene_pool',
 'health_science',
 'hicnet_medical',
 'ivf_et',
 'medical_newsletter',
 'metropolitan_area',
 'mmwr_date',
 'newsletter_page',
 'safety_belt',
 'tucson_arizona',
 'year_olds',
 'assault_weapon',
 'nntp_post',
 'video_card',
 'works_fine',
 'stand_alone',
 'craig_williamson',
 'apr_gmt',
 'news_reader',
 'bobby_mozumder',
 'mats_andtbacka',
 'postings_uninc',
 'unorganized_usenet',
 'nntp_post',
 'greatly_appreciat',
 'ncar_boulder',
 'donald_boell',
 'nntp_post',
 'distribution_world',
 'plus_shipp',
 'hard_drive',
 'login_gu',
 'best_off',
 'access_unix',
 'tape_backup',
 'ethernet_card',
 'nntp_post',
 'half_hour',
 'andy_woodward',
 'wales_aberystwyth',
 'nntp_post',
 'distribution_inet',
 'instruction_set',
 'computing_centre',
 '

...

 'cs_dept',
 'edward_ted',
 'ithaca_ny',
 'major_league',
 'jiann_ming',
 'rickey_henderson',
 'clear_waiver',
 'nntp_post',
 'mailing_list',
 'digital_equipment',
 'stuff_deleted',
 'stuff_delet',
 'net_etiquette',
 'tom_testagrossa',
 'nntp_post',
 'colorado_boulder',
 'rintintin_colorado',
 'purdue_university',
 'big_brother',
 'jim_jone',
 'john_redelfs',
 'next_mormon',
 'whos_next',
 'herman_rubin',
 'get_rid',
 'ancient_mayan',
 'phil_fraer',
 'southwestern_louisiana',
 'televison_repo',
 'weeks_ago',
 'tank_bag',
 'parking_lot',
 'nntp_post',
 'serial_number',
 'encryption_scheme',
 'illegal_cipher',
 'system_transmit',
 'host_camelot',
 'nntp_post',
 'distribution_world',
 'autoexec_bat',
 'config_sy',
 'recently_purchas',
 'config_sys',
 'toshiba_cdrom',
 'adaptec_scsi',
 'aspi_do',
 'nntp_post',
 'ohio_state',
 'acs_ohio',
 'works_fine',
 'plain_silly',
 'ryan_scharfy',
 'recreational_drug',
 'december_starr',
 'nntp_post',
 'earl_wallace',
 'self_defense',
 'getting_rid',
 

 'nntp_post',
 'grad_student',
 'ecst_csuchico',
 'apr_gmt',
 'best_offer',
 'applied_mathematic',
 'joseph_askew',
 'original_poster',
 'cal_poly',
 'distribution_usa',
 'law_enforcement',
 'wall_street',
 'national_guard',
 'armored_vehicle',
 'nntp_post',
 'po_andrew',
 'engineering_carnegie',
 'mellon_pittsburgh',
 'freshman_electrical',
 'distribution_usa',
 'religious_war',
 'microsoft_corp',
 'gay_bash',
 'bob_sarver',
 'nntp_post',
 'distribution_usa',
 'nntp_post',
 'distribution_world',
 'disclaimers_apply',
 'voice_fax',
 'nec_fg',
 'ati_graphic',
 'nntp_post',
 'host_enterpoop',
 'gerard_odriscoll',
 'subwindow_mode',
 'hard_drive',
 'config_sys',
 'himem_sys',
 'maxtor_lxt',
 'adaptec_scsi',
 'aspi_do',
 'new_york',
 'standard_disclaimer',
 'mvp_surprise',
 'nhl_award',
 'worst_opinion',
 'joseph_stiehm',
 'paying_attention',
 'rangers_messi',
 'nntp_post',
 'theodore_chen',
 'station_wagon',
 'blah_blah',
 'nntp_post',
 'california_berkeley',
 'garnet_berkeley',
 'nntp_po

 'nazi_collaboration',
 'aryan_race',
 'delco_electronic',
 'electronics_kokomo',
 'chinese_restaurant',
 'msg_sensitivity',
 'steve_dyer',
 'original_poster',
 'intellectually_dishonest',
 'consulting_cambridge',
 'harvard_rayssd',
 'linus_spdcc',
 'blah_blah',
 'amino_acid',
 'essential_amino',
 'brain_re',
 'dianne_murray',
 'read_olney',
 'peer_review',
 'mothers_milk',
 'intra_ventricular',
 'peer_reviewed',
 'nntp_post',
 'henry_spencer',
 'alaska_fairbank',
 'michael_adam',
 'launch_pad',
 'moonbase_race',
 'doug_mohney',
 'sounding_rocket',
 'nntp_post',
 'california_institute',
 'dumbest_automotive',
 'nntp_post',
 'public_domain',
 'husc_harvard',
 'newsreader_tin',
 'nntp_post',
 'incredible_hulk',
 'marvel_comic',
 'mint_condition',
 'near_mint',
 'alpha_flight',
 'new_mutant',
 'silver_surfer',
 'tuesday_april',
 'star_trek',
 'appearance_sabretooth',
 'circa_ufl',
 'green_goblin',
 'grey_hulk',
 'hulk_st',
 'keown_art',
 'liefeld_bagged',
 'miller_art',
 'omega_man',
 'ro

 'nntp_post',
 'express_access',
 'online_communication',
 'davis_nicoll',
 'nntp_post',
 'jewish_baseball',
 'yasser_arafat',
 'syrias_expansion',
 'adam_shostack',
 'aiken_computation',
 'john_perry',
 'lab_harvard',
 'sexual_favor',
 'wouldnt_waste',
 'southern_lebanon',
 'gulf_war',
 'jordan_river',
 'syrian_troop',
 'israels_occupation',
 'negotiating_table',
 'nntp_post',
 'nntp_posting',
 'apr_gmt',
 'plus_minus',
 'london_ontario',
 'western_ontario',
 'bob_gainey',
 'uwo_ca',
 'newsgroups_rec',
 'sport_hockey',
 'date_fri',
 'netcom_online',
 'communication_service',
 'nntp_post',
 'opinions_express',
 'space_shuttle',
 'dillon_pyron',
 'dseg_ti',
 'host_skndiv',
 'lewisville_vax',
 'sender_unless',
 'ti_dseg',
 'gestures_padi',
 'home_texan',
 'robin_hood',
 'computer_science',
 'std_disclaimer',
 'cookamunga_tourist',
 'kent_sandvik',
 'alink_ksand',
 'cheers_kent',
 'private_activitie',
 'saving_grace',
 'mohammad_razi',
 'david_weisberger',
 'saratoga_spring',
 'skidmore_c

 'david_sternlight',
 'dsi_uscrpac',
 'omissions_except',
 'wiretap_chip',
 'escrow_agencie',
 'hard_core',
 'hackers_spook',
 'derek_atkin',
 'deposit_box',
 'nazi_germany',
 'distribution_usa',
 'standard_disclaimer',
 'rodney_k',
 'says_moop',
 'ted_frank',
 'police_officer',
 'koppenhoefer_kyle',
 'refused_submit',
 'nntp_post',
 'last_night',
 'carnegie_mellon',
 'pittsburgh_pa',
 'po_andrew',
 'software_engineer',
 'doug_gilmour',
 'cherry_coach',
 'administrative_comput',
 'nntp_post',
 'colorado_boulder',
 'spot_colorado',
 'san_francisco',
 'dod_ama',
 'marco_seirio',
 'nntp_post',
 'distribution_world',
 'top_ten',
 'nntp_post',
 'opinions_express',
 'heat_shock',
 'barbecued_food',
 'health_risk',
 'nntp_post',
 'computer_science',
 'newsreader_tin',
 'cs_dept',
 'press_conference',
 'springfield_indian',
 'math_comp',
 'online_communication',
 'san_diego',
 'high_tech',
 'distribution_world',
 'new_york',
 'non_existent',
 'extermination_ohanus',
 'longer_exist',
 'might_se

In [28]:
id2word[0]

'addition'

In [29]:
# Human readable format of corpus (term-frequency)
[[(id2word[id], freq) for id, freq in cp] for cp in corpus[:1]]

[[('addition', 1),
  ('anyone', 2),
  ('body', 1),
  ('bricklin', 1),
  ('bring', 1),
  ('call', 1),
  ('car', 5),
  ('could', 1),
  ('day', 1),
  ('door', 2),
  ('early', 1),
  ('engine', 1),
  ('enlighten', 1),
  ('front_bumper', 1),
  ('funky', 1),
  ('history', 1),
  ('host', 1),
  ('info', 1),
  ('know', 1),
  ('late', 1),
  ('lerxst', 1),
  ('line', 1),
  ('look', 2),
  ('mail', 1),
  ('make', 1),
  ('maryland_college', 1),
  ('model', 1),
  ('name', 1),
  ('neighborhood', 1),
  ('nntp_post', 1),
  ('organization', 1),
  ('park', 1),
  ('production', 1),
  ('rac_wam', 1),
  ('really', 1),
  ('rest', 1),
  ('s', 1),
  ('see', 1),
  ('separate', 1),
  ('small', 1),
  ('spec', 1),
  ('sport', 1),
  ('tellme', 1),
  ('thank', 1),
  ('thing', 1),
  ('umd', 1),
  ('university', 1),
  ('whatev', 1),
  ('where', 1),
  ('wonder', 1),
  ('year', 1)]]

In [30]:
%%time
# This cell takes about 4 minutes to run on my machine.  -j

# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=7, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

CPU times: user 2min 47s, sys: 15.5 s, total: 3min 3s
Wall time: 2min


In [31]:
# Print the Keyword in the 10 topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.017*"key" + 0.016*"file" + 0.011*"use" + 0.011*"system" + '
  '0.010*"information" + 0.010*"code" + 0.008*"program" + 0.008*"available" + '
  '0.007*"bit" + 0.006*"display"'),
 (1,
  '0.011*"space" + 0.006*"information" + 0.006*"chip" + 0.005*"also" + '
  '0.005*"include" + 0.005*"research" + 0.004*"may" + 0.004*"science" + '
  '0.004*"system" + 0.004*"reference"'),
 (2,
  '0.358*"ax" + 0.026*"max" + 0.014*"window" + 0.009*"card" + 0.005*"color" + '
  '0.005*"graphic" + 0.004*"drive" + 0.004*"version" + 0.004*"board" + '
  '0.004*"server"'),
 (3,
  '0.028*"game" + 0.027*"team" + 0.015*"player" + 0.014*"win" + 0.013*"play" + '
  '0.012*"hockey" + 0.011*"year" + 0.010*"season" + 0.008*"nhl" + '
  '0.007*"goal"'),
 (4,
  '0.015*"say" + 0.013*"not" + 0.011*"people" + 0.010*"would" + 0.007*"think" '
  '+ 0.007*"make" + 0.007*"believe" + 0.007*"christian" + 0.007*"do" + '
  '0.006*"god"'),
 (5,
  '0.013*"armenian" + 0.011*"government" + 0.011*"gun" + 0.008*"people" + '
  '0.008*"st

In [32]:
%%time

# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)
print('\n')


Perplexity:  -8.666552095303281

Coherence Score:  0.6099308718182977


CPU times: user 14.4 s, sys: 1.98 s, total: 16.4 s
Wall time: 20.7 s


In [33]:
%%time

# Visualize the topics
#
# If you get an error like this: "pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. 
# A future version of pandas will change to not sort by default."
#
# then from the command line do: "pip install pandas==0.21.0"

pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)  # sort=False ? sort=True
vis
print('\n')



CPU times: user 7.35 s, sys: 1.74 s, total: 9.09 s
Wall time: 8.9 s


In [34]:
#import pandas as pd
#pd.__version__

vis

PreparedData(topic_coordinates=            Freq  cluster  topics         x         y
topic                                                
6      42.312149        1       1  0.244131 -0.057651
4      16.323725        1       2  0.249345 -0.013578
1      12.254162        1       3  0.042152 -0.010321
2      11.041905        1       4 -0.295933 -0.288585
5       8.065489        1       5 -0.012672  0.114009
0       6.329808        1       6 -0.001091 -0.036446
3       3.672769        1       7 -0.225932  0.292573, topic_info=      Category          Freq           Term         Total  loglift  logprob
term                                                                       
4146   Default  63714.000000             ax  63714.000000  30.0000  30.0000
21     Default  13138.000000           line  13138.000000  29.0000  29.0000
3721   Default   4620.000000            max   4620.000000  28.0000  28.0000
82     Default  17601.000000            not  17601.000000  27.0000  27.0000
30     Default  11742.000000   organization  11742.000000  26.0000  26.0000
303    Default   7251.000000            say   7251.000000  25.0000  25.0000
118    Default   7370.000000            get   7370.000000  24.0000  24.0000
65     Default  11273.000000             do  11273.000000  23.0000  23.0000
411    Default   5302.000000         people   5302.000000  22.0000  22.0000
154    Default  12420.000000          would  12420.000000  21.0000  21.0000
50     Default   3856.000000           year   3856.000000  20.0000  20.0000
2726   Default   1662.000000           game   1662.000000  19.0000  19.0000
322    Default  10039.000000          write  10039.000000  18.0000  18.0000
215    Default   2433.000000         window   2433.000000  17.0000  17.0000
317    Default   4843.000000            use   4843.000000  16.0000  16.0000
567    Default   1580.000000           team   1580.000000  15.0000  15.0000
738    Default   2047.000000           file   2047.000000  14.0000  14.0000
756    Default   1695.000000            key   1695.000000  13.0000  13.0000
1911   Default   2147.000000          space   2147.000000  12.0000  12.0000
18     Default   7748.000000           know   7748.000000  11.0000  11.0000
5225   Default   1731.000000       armenian   1731.000000  10.0000  10.0000
206    Default   3574.000000         system   3574.000000   9.0000   9.0000
244    Default   8684.000000             be   8684.000000   8.0000   8.0000
55     Default   7915.000000        article   7915.000000   7.0000   7.0000
2411   Default   2229.000000    information   2229.000000   6.0000   6.0000
147    Default   5589.000000          think   5589.000000   5.0000   5.0000
29     Default   4649.000000      nntp_post   4649.000000   4.0000   4.0000
1362   Default   1945.000000     government   1945.000000   3.0000   3.0000
60     Default   2092.000000           card   2092.000000   2.0000   2.0000
16     Default   4298.000000           host   4298.000000   1.0000   1.0000
...        ...           ...            ...           ...      ...      ...
1531    Topic7    300.227997            bus    301.173370   3.3011  -5.2847
1967    Topic7    280.149292       canadian    281.086212   3.3009  -5.3539
4073    Topic7    263.958679          radio    264.901093   3.3007  -5.4135
12603   Topic7    256.313354            mon    257.247345   3.3006  -5.4428
6345    Topic7    246.997955           wing    247.937408   3.3004  -5.4799
8227    Topic7    241.859436      vancouver    242.795380   3.3004  -5.5009
4071    Topic7    237.818146        morning    238.773361   3.3002  -5.5177
1974    Topic7    227.542007        playoff    228.474945   3.3001  -5.5619
17009   Topic7    229.355850            pit    230.297089   3.3001  -5.5540
51362   Topic7    217.396469            nyr    218.329254   3.2999  -5.6075
8061    Topic7    210.299683             la    211.237137   3.2998  -5.6407
4813    Topic7    208.495422            min    209.430939   3.2997  -5.6493
3169    Topic7    204.325516     homosexual    205.